In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Unmodified Scores

In [0]:
systems = ['PBMT-R', 'Hybrid', 'EncDecA', 'Dress', 'Dress-Ls']
fkgl = []
bleu = []
sari = []

import os
os.chdir('/content/drive/My Drive/Metrics/')
corpus = 'Newsela'

for system in systems:
  path = "./dress/all-system-output/" + corpus + "/test/" + system
  fkgl.append(FKGL(path))
  bleu.append(BLEU(corpus, path))
  sari.append(SARI(corpus, path))

# Results Table

In [0]:
import os
frames = []
experiments = [1,2,3,4,5,6]

for experiment in experiments:
  os.chdir('/content/drive/My Drive/Metrics/new_results/approach'+str(experiment))

  import pandas as pd
  corpus = 'Newsela'
  system = 'EncDecA' #'Dress-Ls'
  frame = pd.DataFrame(columns = ['percent', 'fkgl', 'bleu', 'sari'])
  #percent = 10
  #while percent <= 100:
  percents = [10, 50, 100]
  for percent in percents:
    data = pd.read_csv(corpus + system + str(int(percent)) + '.csv')
    data = data.drop(data.columns[0], axis=1) # drop indices
    row = pd.Series([percent], index=['percent']).append(data.mean())
    frame = frame.append(row, ignore_index=True)
    #percent += 10
  frames.append(frame)

In [0]:
os.chdir('/content/drive/My Drive/Metrics/new_results')
pd.concat(frames, axis=1).to_csv('results_EncDecA.csv')

# Modify Output

In [0]:
import os
os.chdir('/content/drive/My Drive/Metrics/')

corpus = 'Newsela'

systems = ['EncDecA', 'Hybrid']
for i in range(1,7):
  for system in systems:
    path = "./dress/all-system-output/" + corpus + "/test/" + system
    f = open(path, mode="r")
    contents = f.read().splitlines()
    numsentences = len(contents)
    print(numsentences)
    run(corpus, system, contents, numsentences, i) # overwrite
    #x1,x4 = detailed(corpus, system, contents, numsentences, 3)

  '''
  1-random-period
  2-random-the
  3-replace-longest
  4-replace-rand-period
  5-replace-rand-the
  6-random-longest
  '''

## Detailed Stats for SARI


*   Require generate() and SARI() functions as well
*   "detailed" returns the 6 precision-recall values of 1-gram and 4-gram
*   x1,x4 = detailed(corpus, system, contents, numsentences, 2)

In [0]:
print(F1(x1[0], x1[1]))
print(F1(x1[2], x1[3]))
print(x1[4])
print(F1(x4[0], x4[1]))
print(F1(x4[2], x4[3]))
print(x4[4])

0.0504870466321
0.118097187852
0.9722
0.0150006557377
0.0462858108108
0.9878


In [0]:
def F1(P, R):
  return 2*(P*R)/(P+R)

In [0]:
def detailed(corpus, system, contents, numsentences, approach):
  """run experiment on given data with specified approach"""

  percent = 1
  generate(approach, corpus, system, contents, numsentences, percent)
  return SARI_stats(corpus, system, percent)

In [0]:
def SARI_stats(corpus, system, percent):
  stats1 = [0,0,0,0,0,0]
  stats4 = [0,0,0,0,0,0]
  for n in range(10):
    path = './100files/' + str(n) + '.txt'
    out = SARI(corpus, path)
    stats1 = [x + y for x, y in zip(stats1, out[1])]
    stats4 = [x + y for x, y in zip(stats4, out[4])]
  stats1 = [x/10 for x in stats1]
  stats4 = [x/10 for x in stats4]
  return stats1, stats4

## Actual Experiments

In [0]:
def run(corpus, system, contents, numsentences, approach):
  """run experiment on given data with specified approach"""

  percents = [0.1, 0.5, 1] #0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9, 
  for percent in percents:
    generate(approach, corpus, system, contents, numsentences, percent)
    evaluate(approach, corpus, system, percent)

In [0]:
def generate(approach, corpus, system, contents, numsentences, percent):
  """generate 100 files with specified percent sentences modified"""

  from random import shuffle, randint

  with open('./100files/current.txt', 'w+') as file:
    file.write(str(approach)+'\n' + corpus + '\n' + system + '\n' + str(percent))

  for n in range(100): # repeat 100 times for each corpus/system
    markers = [1]*int(numsentences*percent) + [0]*(numsentences-int(numsentences*percent))
    shuffle(markers)

    with open('./100files/' + str(n) + '.txt', 'w+') as file:
      for i in range(numsentences):
        sentence = contents[i]
        words = contents[i].split(' ')
        if markers[i] == 1 and len(words) > 1:
          sentence = ""
          if(approach == 0):
            sentence = contents[i]
          elif(approach == 1):      # random-period
            where = randint(1,len(words)-2) # at least 1 word in both halves
            for j in range(len(words)):
              if j == where:
                sentence += ". "
              sentence += words[j] + " "
          elif(approach == 2):    # random-the
            where = randint(1,len(words)-1)
            for j in range(len(words)):
              if j == where:
                sentence += "the "
              sentence += words[j] + " "
          elif(approach == 3):    # replace-longest
            where = find_longest(words)
            for j in range(len(words)):
              if j == where:
                sentence += "the "
              else:
                sentence += words[j] + " "
          elif(approach == 4):    # replace-rand-period
            where = randint(1,len(words)-1)
            for j in range(len(words)):
              if j == where:
                sentence += ". "
              else:
                sentence += words[j] + " "
          elif(approach == 5):    # replace-rand-the
            where = randint(1,len(words)-2)
            for j in range(len(words)):
              if j == where:
                sentence += "the "
              else:
                sentence += words[j] + " "
          else:                   # 6 = random-longest
            longest = find_longest(words)
            where = randint(1,len(words)-1)
            for j in range(len(words)):
              if j == where:
                  sentence += ". "
              if j == longest:
                sentence += "the "
              else:
                sentence += words[j] + " "
        file.write(sentence + '\n')

In [0]:
def find_longest(words):
  """Return index of the longest word in the given list"""
  maxLength = 0
  index = -1
  for i in range(len(words)):
    if len(words[i]) > maxLength:
      maxLength = len(words[i])
      index = i
  return index

In [0]:
def evaluate(approach, corpus, system, percent):
  """run 100 rounds and output results to csv"""

  fkgl = []
  bleu = []
  sari = []
  import os
  os.chdir('/content/drive/My Drive/Metrics/')

  for n in range(100):
    path = './100files/' + str(n) + '.txt'
    fkgl.append(FKGL(path))
    bleu.append(BLEU(corpus, path))
    sari.append(SARI(corpus, path))

  # output to file
  import pandas as pd
  '''path = './new_results/approach'+str(approach)+'/' + corpus + system + str(int(percent*100)) + '.csv'
  data = pd.read_csv(path)
  data['fkgl'] = fkgl
  data = data.drop(data.columns[0], axis=1)
  data.to_csv(path)'''
  result = pd.DataFrame(list(zip(fkgl, bleu, sari)), columns = ['fkgl', 'bleu', 'sari'])
  result.to_csv('./new_results/approach'+str(approach)+'/' + corpus + system + str(int(percent*100)) + '.csv')

# Metrics Functions
Run all below before anything else.

##FKGL

In [2]:
# setup

import os
os.chdir('/content/')

!pip install nltk
import nltk
nltk.download('punkt')

import os
os.chdir('/content/drive/My Drive/Metrics/readability')
import readability

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def FKGL(path): # absolute path
    import io
    import sys
    
    reload(sys)
    sys.setdefaultencoding('utf8')

    f = io.open(path, mode="r", encoding="utf-8")
    contents = f.read()
    rd = readability.Readability(contents)
    return rd.FleschKincaidGradeLevel()

In [0]:
x = FKGL('/content/drive/My Drive/Metrics/dress/all-system-output/Newsela/test/Dress-Ls')

In [5]:
x

5.0241

##BLEU

In [0]:
import os
os.chdir('/content/drive/My Drive/Metrics/')

!chmod 755 '/content/drive/My Drive/Metrics/ppdb-simplification-release-joshua5.0/joshua/bin/bleu'

In [0]:
def BLEU(corpus, path):
  import subprocess

  if corpus == 'WikiLarge':
    # given path, convert to .lower first
    !python tolower.py path path
    pipe = subprocess.Popen("./ppdb-simplification-release-joshua5.0/joshua/bin/bleu " + path + " ./simplification/data/turkcorpus/test.8turkers.tok.turk 8", shell=True, stdout=subprocess.PIPE).stdout
    output = pipe.read()
    out = output.split('\n')
    return float(out[len(out)-3].split('=')[2][1:])
  
  else:
    ref = " ./dress/all-system-output/" + corpus + "/test/Reference 1"
    pipe = subprocess.Popen("./ppdb-simplification-release-joshua5.0/joshua/bin/bleu " + path + ref, shell=True, stdout=subprocess.PIPE).stdout
    output = pipe.read()
    out = output.split('\n')
    '''precisions = []
    pre = output.split('BLEU_precision')
    for i in range(4):
      precisions.append(float(pre[i+1].split('= ')[2]))'''
    return float(out[len(out)-3].split('=')[2][1:]) #, precisions

##SARI

In [0]:
import os
os.chdir('/content/drive/My Drive/Metrics/')

In [9]:
%%shell
chmod 755 ./ppdb-simplification-release-joshua5.0/joshua/bin/star
chmod 755 ./ppdb-simplification-release-joshua5.0/joshua/bin/star_1
chmod 755 wikilarge.show_all_sari.sh
chmod 755 star

export JAVA_HOME=/usr/lib/jvm/java-1.8.0
export JOSHUA=/disk/scratch/Software/joshua_5/ppdb-simplification-release-joshua5.0/joshua

export LC_ALL=en_US.UTF-8
export LANG=en_US.UTF-8

In [0]:
def SARI(corpus, path):
  import subprocess

  if corpus == 'WikiLarge':
    # doesn't work yet no idea why
    subprocess.Popen("tolower.py" + path +" "+" "+ path, shell=True)
    orig = " ./simplification/data/turkcorpus/test.8turkers.tok.norm"
    ref = " ./simplification/data/turkcorpus/test.8turkers.tok.turk"
    pipe = subprocess.Popen("./ppdb-simplification-release-joshua5.0/joshua/bin/star " + path + ref + orig, shell=True, stdout=subprocess.PIPE).stdout
    output = pipe.read()
    print(output)
    out = output.split('STAR = ')
    return float(out[1][:6])
  
  else:
    ref = " ./dress/all-system-output/" + corpus + "/test/Reference"
    orig = " ./dress/all-system-output/" + corpus + "/test/Complex"

    pipe = subprocess.Popen("./ppdb-simplification-release-joshua5.0/joshua/bin/star_1 " + path + ref + orig, shell=True, stdout=subprocess.PIPE).stdout
    output = pipe.read()
    out = output.split('STAR = ')
    score = float(out[1][:6])

    '''outputs = [[0 for x in range(6)] for y in range(5)] 
    for i in range(1,5):
      ngram = output.split(str(i)+'-gram = ')[1]
      for j in range(6):
        outputs[i][j] = float(ngram.split(' ')[j][:6])'''
    return score #outputs

In [11]:
# Check that all metrics are not throwing any error

#FKGL('./100files/0.txt')
#BLEU('Newsela', './100files/0.txt')
SARI('Newsela', './100files/0.txt')

0.347